In [25]:
import pandas
pandas.set_option("display.max_rows", 50, "display.max_columns", 50)

import numpy
import pygsheets

import Pipelines_Current dataset

In [26]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(client_secret=credentials_directory+'client_secret.json')
spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek')

#spreadsheet[1] "Gas Pipelines" tab is the second index
gas_pipes = spreadsheet[1].get_as_df(encoding='latin1')
oil_pipes = spreadsheet[2].get_as_df(encoding='latin1')
#owners = spreadsheet[2].get_as_df()

gas_pipes = gas_pipes.drop('WKTFormat', axis=1) # delete WKTFormat column
oil_pipes = oil_pipes.drop('WKTFormat', axis=1)
pipes_df_orig = pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#pipe = spreadsheet[1].get_as_df()
#pipe = pipe.drop('WKTFormat', axis=1)

#get other relevant sheets
country_ratios_df = spreadsheet[5].get_as_df(encoding='latin1')
owners_df_orig = spreadsheet[3].get_as_df(encoding='latin1')
owners_df_orig.set_index('ProjectID', inplace=True)

In [27]:
country_ratios_df.replace('--', numpy.nan, inplace=True)

owners_df_orig.replace('',numpy.nan,inplace=True)
owners_df_orig.replace('--',numpy.nan,inplace=True)

pipes_df_orig.replace('--',numpy.nan,inplace=True)

In [28]:
region_df_orig = spreadsheet[4].get_as_df(encoding='latin1')

In [29]:
region_df_eu = region_df_orig.copy()[region_df_orig['EuropeanUnion']=='Yes']
region_df_egt = region_df_orig.copy()[region_df_orig['EuroGasTracker']=='Yes']
region_df_europe = region_df_orig.copy()[region_df_orig['Region']=='Europe']
region_df_eu_uk = region_df_orig.copy()[(region_df_orig['Region']=='Europe') | 
                                          (region_df_orig['Country'].isin(['United Kingdom','Israel']))]

In [30]:
region_df_touse = region_df_egt.copy()

### create EU-specific dataframes for region, country_ratios_df, owners_df

In [24]:
country_ratios_df_touse = country_ratios_df.copy()[country_ratios_df['Country'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

owners_df_touse = owners_df_orig.copy()[owners_df_orig['Countries'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

pipes_df_touse = pipes_df_orig.copy()[pipes_df_orig['Countries'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

### set up info to analyze Owners tab

In [ ]:
# get a set list of all owners
owner_column_list = ['Owner1', 
                     'Owner2', 
                     'Owner3', 
                     'Owner4', 
                     'Owner5', 
                     'Owner6', 
                     'Owner7', 
                     'Owner8', 
                     'Owner9', 
                     'Owner10', 
                     'Owner11']

percent_column_list = ['Percent1', 
                       'Percent2', 
                       'Percent3', 
                       'Percent4', 
                       'Percent5', 
                       'Percent6', 
                       'Percent7', 
                       'Percent8', 
                       'Percent9', 
                       'Percent10', 
                       'Percent11']

owner_id_list = ['ID1', 
                 'ID2', 
                 'ID3', 
                 'ID4', 
                 'ID5', 
                 'ID6', 
                 'ID7', 
                 'ID8', 
                 'ID9', 
                 'ID10', 
                 'ID11']

### sum MergedKmByCountry and MergedKmByRegion

In [ ]:
status_list = ['Proposed', 
               'Construction', 
               'Shelved', 
               'Cancelled', 
               'Operating', 
               'Idle', 
               'Mothballed', 
               'Retired']
country_list = sorted(list(set(country_ratios_df_touse['Country'])))
region_list = sorted(list(set(country_ratios_df_touse['Region'])))

In [ ]:
excel_status_list = ['Proposed', 
                     'Construction', 
                     'In Development (Proposed + Construction)', 
                     'Shelved', 
                     'Cancelled', 
                     'Operating', 
                     'Idle', 
                     'Mothballed', 
                     'Retired']

In [ ]:
fuel_type = 'Gas'
#fuel_type = 'Oil'
#fuel_type = 'NGL'

country_ratios_df_subset = country_ratios_df_touse.copy()[country_ratios_df_touse['Fuel']==fuel_type]

km_by_country = pandas.DataFrame(columns=status_list, index=country_list)
km_by_region = pandas.DataFrame(columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_country[status] = country_ratios_df_subset_status.groupby('Country')['MergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_region[status] = country_ratios_df_subset_status.groupby('Region')['MergedKmByCountry'].sum()

# fille NaN with 0.0
km_by_region = km_by_region.fillna(0)
km_by_country = km_by_country.fillna(0)

km_by_region['In Development (Proposed + Construction)'] = km_by_region[['Proposed','Construction']].sum(axis=1)
km_by_country['In Development (Proposed + Construction)'] = km_by_country[['Proposed','Construction']].sum(axis=1)

km_by_country = km_by_country[excel_status_list]
km_by_region = km_by_region[excel_status_list]

km_by_region.index.name = 'Region'
km_by_country.index.name = 'Country'

totals_row = km_by_region.sum(axis=0)
totals_row.name = 'Total'
km_by_region = km_by_region.append(totals_row)

totals_row = km_by_country.sum(axis=0)
totals_row.name = 'Total'
km_by_country = km_by_country.append(totals_row)

km_by_region.to_excel('EuroGas'+'KmByRegionAndStatus.xlsx')
km_by_country.to_excel('EuroGas'+'KmByCountryAndStatus.xlsx')

### create an excel file with all unique owners, not specific to fuel type

In [ ]:
owner_list = []
for column in owner_column_list:
    owner_list+=list(owners_df_orig[column])
owner_list = list(set(owner_list))
# remove empty (no owner)
owner_list.remove(numpy.nan)
unique_owner_list = sorted(owner_list)

#pandas.Series(unique_owner_list).to_excel('AllFuels'+'UniqueOwnersList.xlsx')

## pipeline km by parent company (owner) and project status

In [ ]:
fuel_type = 'Gas'
#fuel_type = 'Oil'
#fuel_type = 'NGL'

owners_df_subset = owners_df_touse.copy()[owners_df_touse['Fuel']==fuel_type]

##################################################

owner_list_subset = []
for column in owner_column_list:
    owner_list_subset+=list(owners_df_subset[column])
owner_list_subset = list(set(owner_list_subset))
# remove empty (no owner)
owner_list_subset.remove(numpy.nan)
unique_owner_list_subset = sorted(owner_list_subset)

##################################################
# convert percents to fractions
##################################################
owners_df_fractions = owners_df_subset.copy()

for col in percent_column_list:
    owners_df_fractions[col] = owners_df_fractions[col].apply(lambda x: numpy.nan if x in [numpy.nan] 
                                          else x[:-1]).astype(float)/100
#df['col'] = df['col'].apply(lambda x: np.nan if x in ['-'] else x[:-1]).astype(float)/100

##################################################
# create km count by owner, status
##################################################
no_owner_info_count = 0
owners_km_by_status_df = pandas.DataFrame(0, index=unique_owner_list_subset, columns=status_list)

for status in status_list:
    
    owners_df_temporary = owners_df_fractions.copy()[owners_df_fractions['Status']==status]
    
    for idx,row in owners_df_temporary.iterrows():
        
        ### how many owners are there?
        row_owners = list(row[owner_column_list])
        row_owners = [i for i in row_owners if str(i)!='nan']
        n_owners = row_owners.__len__()
        row_fractions = list(row[percent_column_list])
        row_fractions = [i for i in row_fractions if str(i)!='nan']

        ### if there are no owners listed, continue to next loop iteration
        if n_owners==0:
            no_owner_info_count+=1
            continue # doesn't complete the rest of the ifs

        ### now if row fractions is an empty list, create equal fractions list instead
        if row_fractions==[]:
            row_fractions = [1/n_owners]*n_owners

        pipe_length = row['LengthMergedKm']
        owner_km_fractions = numpy.array(row_fractions)*pipe_length

        for owner_idx in range(n_owners):
            owners_km_by_status_df.loc[row_owners[owner_idx]][status]+=owner_km_fractions[owner_idx]

owners_km_by_status_df.index.name = 'Parent Company'
owners_km_by_status_df['In Development (Proposed + Construction)'] = owners_km_by_status_df[['Proposed','Construction']].sum(axis=1)
owners_km_by_status_df = owners_km_by_status_df[excel_status_list]

# rearrange the order of the columns for output
owners_km_by_status_df = owners_km_by_status_df[excel_status_list]

totals_row = owners_km_by_status_df.sum(axis=0)
totals_row.name = 'Total'
owners_km_by_status_df = owners_km_by_status_df.append(totals_row)

owners_km_by_status_df.to_excel('EuroGas'+'KmByOwnerAndStatus.xlsx')

### pipeline km by start year, type

In [ ]:
pipes_started_eu = pipes_df_touse.copy()
#pipes_started_eu['StartYearLatest'].replace(numpy.nan,'',inplace=True)

pipes_started_eu = pipes_started_eu[(pipes_started_eu['Status'].isin(['Operating'])) &
                              (pipes_started_eu['Fuel']=='Gas')]
pipes_started_eu_sum = pipes_started_eu.groupby('StartYearLatest')['LengthMergedKm'].sum()

In [ ]:
km_by_start_year = pandas.DataFrame(index=list(range(1980,2022)), columns=['Gas pipeline km'])
km_by_start_year.index.name = 'Start year'

km_by_start_year['Gas pipeline km'] = pipes_started_eu_sum
km_by_start_year.replace(numpy.nan,0,inplace=True)

totals_row = km_by_start_year.sum(axis=0)
totals_row.name = 'Total'
km_by_start_year = km_by_start_year.append(totals_row)

km_by_start_year.to_excel('EuroGas'+'KmByStartYear.xlsx')